In [ ]:
from typing import *

import numpy as np
import matplotlib.pyplot as plt
import torch

from tqdm import tqdm
from xai.genome import Genome
from xai.bytes import GigaBytes
from xai.policy import Policy
from xai.asteroids import Asteroids
from xai.stream import Stream

In [ ]:
import random

random.normalvariate(sigma=-1), random.normalvariate(sigma=1)
torch.cuda.empty_cache()

In [ ]:
genome = Genome(translate=True, rotate=True, volatility=1.0)

population = genome.populate(
    population_size=5,
    use_ram_genomes=False,
    use_ram_observations=True,
    max_memory=GigaBytes(5.0),
    max_observation_memory=GigaBytes(20.0),
    subprocesses_observation_memory_size=GigaBytes(5.0),
    verbose=True
)

In [ ]:
population.evolve(
    generations=50,
    survivors_cnt=0,
    elites_cnt=0,
    roulettes_cnt=0,
    random_cnt=0,
    rank_cnt=15,
    checkpoints_directory="Test",
    number_of_processes=4
)